In [227]:
import alpaca_trade_api as alpaca
from alpaca.trading.client import TradingClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.timeframe import TimeFrame
from dotenv import load_dotenv
import yfinance as yf
import os
import pandas as pd
import numpy as np
import datetime as dt
import time
import requests
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
import joblib

In [228]:
paper = True

# initialize API from API keys in .env
load_dotenv()

if paper:
    api_key = os.environ['APCA-API-PAPER-KEY-ID']
    api_secret_key = os.environ['APCA-API-PAPER-SECRET-KEY']
    api_base_url = 'https://paper-api.alpaca.markets'
else:
    api_key = os.environ['APCA-API-KEY-ID']
    api_secret_key = os.environ['APCA-API-SECRET-KEY']
    api_base_url = 'https://api.alpaca.markets'

api = alpaca.REST(api_key, api_secret_key, api_base_url)
account = api.get_account()
trading_client = TradingClient(api_key, api_secret_key, paper=paper)
data_client = StockHistoricalDataClient(api_key, api_secret_key)

In [229]:
def get_SP500():
    market_caps = []
    SP500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
    #SP500 = SP500[0:20]
    for index, row in SP500.iterrows():
        print(f"{index}: {row['Symbol']}")
        query_attempts = 0
        market_cap = 0
        while (True):
            query_attempts += 1
            if query_attempts >= 3:
                raise Exception('query failed')
            try:
                market_cap = yf.Ticker(row['Symbol']).info.get("marketCap")
                break
            except:
                time.sleep(3)
        market_caps.append(market_cap)
        print(market_cap)
        print()
        time.sleep(1)
    SP500["Market Cap"] = market_caps
    return SP500

def get_SP(n=500):
    return pd.read_csv('SP500.csv')[0:n].sort_values(by='Symbol')

def get_weekly_stock_bars(symbols, start=dt.datetime(2019, 1, 1), filename='output.csv'):
    request_params = StockBarsRequest(
        symbol_or_symbols=symbols,
        timeframe=TimeFrame.Day,
        start=start,
        adjustment='all'
    )
    bars = repeated_get_stock_bars(request_params)
    df = bars.df
    df = df[df.index.get_level_values('timestamp').day_name() == 'Friday']
    df['log return'] = np.log(df['close'] / df['close'].groupby(level=0).shift(1))
    df = df.dropna(subset=['log return'])
    df = df[['close', 'log return']]
    df = df.swaplevel('timestamp', 'symbol')
    df = df.sort_index(level=['timestamp', 'symbol'], ascending=[True, True])

    valid_symbols = df.groupby('symbol').size()
    valid_symbols = valid_symbols[valid_symbols == df.index.get_level_values(level=0).nunique()].index
    df = df[df.index.get_level_values(level=1).isin(valid_symbols)]
    df.to_csv(filename)

def repeated_get_stock_bars(request_params):
    query_attempts = 0
    while (True):
        query_attempts += 1
        if query_attempts >= 3:
            raise Exception('query failed')
        try:
            return data_client.get_stock_bars(request_params)
        except:
            time.sleep(3)

def train_and_store_models(test_start_date='2022-01-01', test_end_date='2023-01-01'):
    df = pd.read_csv('output.csv')
    df_pivot = df.pivot(index='timestamp', columns='symbol', values='log return')
    df_test_x = df_pivot[(df_pivot.index >= test_start_date) & (df_pivot.index <= test_end_date)]
    df_train_x = df_pivot[(df_pivot.index < test_start_date) | (df_pivot.index > test_end_date)]
    df_test_y = df_test_x.copy()
    df_train_y = df_train_x.copy()

    for symbol in df['symbol'].unique():
        scaler = StandardScaler()
        df_train_x.loc[:, symbol] = scaler.fit_transform(df_train_x[[symbol]]).astype(df_train_x[symbol].dtype)
        df_test_x.loc[:, symbol] = scaler.transform(df_test_x[[symbol]]).astype(df_test_x[symbol].dtype)

    models = {}
    
    for symbol in df['symbol'].unique():
        print(f"Training {symbol} model...")
        y_train = df_train_y[symbol].values
        X_train = df_train_x.drop(symbol, axis=1)
        y_test = df_test_y[symbol].values
        X_test = df_test_x.drop(symbol, axis=1)

        # Adjusted range for alpha and l1_ratio
        param_grid = {
            'alpha': [0.005, 0.01, 0.025, 0.05, 0.1, 0.25],  # Try smaller alpha values
            'l1_ratio': [0.05, 0.1, 0.2, 0.35, 0.5, 0.65, 0.8, 0.9, 0.95]     # Try a mix between Ridge and Lasso
        }

        # Set up the ElasticNet model
        elasticnet = ElasticNet()

        # Perform grid search with 5-fold cross-validation
        grid_search = GridSearchCV(elasticnet, param_grid, cv=5, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)

        # Best parameters found
        best_alpha = grid_search.best_params_['alpha']
        best_l1_ratio = grid_search.best_params_['l1_ratio']
        print(f"Best alpha: {best_alpha}, Best l1_ratio: {best_l1_ratio}")

        # Fit the ElasticNet model with the best parameters found
        model = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio)
        model.fit(X_train, y_train)
        models[symbol] = model
    joblib.dump(models, 'stock_models.pkl')

def test_model(symbol, test_start_date='2022-01-01', test_end_date='2023-01-01'):
    # Load the models
    loaded_models = joblib.load('stock_models.pkl')

    # Access a specific model
    model = loaded_models[symbol]

    df = pd.read_csv('output.csv')
    df_pivot = df.pivot(index='timestamp', columns='symbol', values='log return')
    df_test_x = df_pivot[(df_pivot.index >= test_start_date) & (df_pivot.index <= test_end_date)]
    df_train_x = df_pivot[(df_pivot.index < test_start_date) | (df_pivot.index > test_end_date)]
    df_test_y = df_test_x.copy()
    df_train_y = df_train_x.copy()

    for symbol2 in df['symbol'].unique():
        scaler = StandardScaler()
        df_train_x.loc[:, symbol2] = scaler.fit_transform(df_train_x[[symbol2]]).astype(df_train_x[symbol2].dtype)
        df_test_x.loc[:, symbol2] = scaler.transform(df_test_x[[symbol2]]).astype(df_test_x[symbol2].dtype)
    
    y_test = df_test_y[symbol].values
    X_test = df_test_x.drop(symbol, axis=1)

    # Assuming `X_train.columns` contains the stock symbols
    coef_df = pd.DataFrame({
        'Stock Symbol': X_test.columns, 
        'Coefficient': model.coef_
    })

    # Sort by absolute value to highlight the most impactful stocks
    coef_df_sorted = coef_df.reindex(coef_df['Coefficient'].abs().sort_values(ascending=False).index)

    print(coef_df_sorted)
    coef_df_sorted.to_csv('model_coefs.csv')
    # Example prediction
    y_pred = model.predict(X_test)
    plt.scatter(x=y_test, y=y_pred)
    plt.plot(y_pred, y_pred, color='red', label='y=x')
    plt.show()
    plt.clf()

def simulate_model(symbol, test_start_date='2022-01-01', test_end_date='2023-01-01'):
    # Load the models
    loaded_models = joblib.load('stock_models.pkl')

    # Access a specific model
    model = loaded_models[symbol]

    df = pd.read_csv('output.csv')
    df_pivot = df.pivot(index='timestamp', columns='symbol', values='log return')
    df_test_x = df_pivot[(df_pivot.index >= test_start_date) & (df_pivot.index <= test_end_date)]
    df_train_x = df_pivot[(df_pivot.index < test_start_date) | (df_pivot.index > test_end_date)]
    df_test_y = df_test_x.copy()
    df_train_y = df_train_x.copy()

    log_return_stds = {}

    for symbol2 in df['symbol'].unique():
        log_return_stds[symbol2] = np.std(df_train_x[symbol2])
        scaler = StandardScaler()
        df_train_x.loc[:, symbol2] = scaler.fit_transform(df_train_x[[symbol2]]).astype(df_train_x[symbol2].dtype)
        df_test_x.loc[:, symbol2] = scaler.transform(df_test_x[[symbol2]]).astype(df_test_x[symbol2].dtype)
    
    y_train = df_train_y[symbol].values
    X_train = df_train_x.drop(symbol, axis=1)
    y_test = df_test_y[symbol].values
    X_test = df_test_x.drop(symbol, axis=1)

    y_pred = model.predict(X_train)
    resid_train = y_train - y_pred
    resid_mean = np.mean(resid_train)
    resid_std = np.std(resid_train)

    beta_sum = sum(abs(model.coef_))

    tests = []
    total_main_pnl = 0
    total_hedge_pnl = 0
    trade_count = 0
    for i in range(len(X_test) - 1):
        timestamp = X_test.index[i]
        y_pred = model.predict(X_test.loc[[timestamp]])[0]
        y_actual = y_test[i]
        z_score = ((y_actual - y_pred) - resid_mean) / resid_std
        tests.append(z_score)

        size = 1000
        
        if z_score >= 2 and z_score < 3:
            main_pnl = 0
            hedge_pnl = 0

            next_timestamp = X_test.index[i+1]
            main_pnl = size * (1 - df[(df['symbol'] == symbol) & (df['timestamp'] == next_timestamp)]['close'].iloc[0] /
                        df[(df['symbol'] == symbol) & (df['timestamp'] == timestamp)]['close'].iloc[0])
            j = 0
            # if i > 0:
            #     print(f"previous price: {df[(df['symbol'] == symbol) & (df['timestamp'] == X_test.index[i-1])]['close'].iloc[0]}")
            # print(f"current price: {df[(df['symbol'] == symbol) & (df['timestamp'] == timestamp)]['close'].iloc[0]}")
            # print(f"next price: {df[(df['symbol'] == symbol) & (df['timestamp'] == next_timestamp)]['close'].iloc[0]}")
            for symbol2 in df['symbol'].unique():
                if symbol == symbol2:
                    continue
                beta = model.coef_[j]
                hedge_weight = beta / beta_sum * log_return_stds[symbol] / log_return_stds[symbol2]
                hedge_pnl += hedge_weight * size * (-1 + df[(df['symbol'] == symbol2) & (df['timestamp'] == next_timestamp)]['close'].iloc[0] /
                        df[(df['symbol'] == symbol2) & (df['timestamp'] == timestamp)]['close'].iloc[0])
                j += 1
            #     if abs(hedge_weight) > 0.1:
            #         print(f"SYMBOL: {symbol2}")
            #         if i > 0:
            #             print(f"previous price: {df[(df['symbol'] == symbol2) & (df['timestamp'] == X_test.index[i-1])]['close'].iloc[0]}")
            #         print(f"current price: {df[(df['symbol'] == symbol2) & (df['timestamp'] == timestamp)]['close'].iloc[0]}")
            #         print(f"next price: {df[(df['symbol'] == symbol2) & (df['timestamp'] == next_timestamp)]['close'].iloc[0]}")
            # print(f"main pnl: {main_pnl}")
            # print(f"hedge pnl: {hedge_pnl}")
            total_main_pnl += main_pnl
            total_hedge_pnl += hedge_pnl
            trade_count += 1

        if z_score <= -2 and z_score > -3:
            main_pnl = 0
            hedge_pnl = 0

            next_timestamp = X_test.index[i+1]
            main_pnl = size * (-1 + df[(df['symbol'] == symbol) & (df['timestamp'] == next_timestamp)]['close'].iloc[0] /
                        df[(df['symbol'] == symbol) & (df['timestamp'] == timestamp)]['close'].iloc[0])
            j = 0
            # if i > 0:
            #     print(f"previous price: {df[(df['symbol'] == symbol) & (df['timestamp'] == X_test.index[i-1])]['close'].iloc[0]}")
            # print(f"current price: {df[(df['symbol'] == symbol) & (df['timestamp'] == timestamp)]['close'].iloc[0]}")
            # print(f"next price: {df[(df['symbol'] == symbol) & (df['timestamp'] == next_timestamp)]['close'].iloc[0]}")
            for symbol2 in df['symbol'].unique():
                if symbol == symbol2:
                    continue
                beta = model.coef_[j]
                hedge_weight = beta / beta_sum * log_return_stds[symbol] / log_return_stds[symbol2]
                hedge_pnl += hedge_weight * size * (1 - df[(df['symbol'] == symbol2) & (df['timestamp'] == next_timestamp)]['close'].iloc[0] /
                        df[(df['symbol'] == symbol2) & (df['timestamp'] == timestamp)]['close'].iloc[0])
                j += 1
                # if abs(hedge_weight) > 0.1:
                #     print(f"SYMBOL: {symbol2}")
                    # if i > 0:
                    #     print(f"previous price: {df[(df['symbol'] == symbol2) & (df['timestamp'] == X_test.index[i-1])]['close'].iloc[0]}")
                    # print(f"current price: {df[(df['symbol'] == symbol2) & (df['timestamp'] == timestamp)]['close'].iloc[0]}")
                    # print(f"next price: {df[(df['symbol'] == symbol2) & (df['timestamp'] == next_timestamp)]['close'].iloc[0]}")
            # print(f"main pnl: {main_pnl}")
            # print(f"hedge pnl: {hedge_pnl}")
            total_main_pnl += main_pnl
            total_hedge_pnl += hedge_pnl
            trade_count += 1

    print(f"main pnl: {total_main_pnl}")
    print(f"hedge pnl: {total_hedge_pnl}")
    print(f"trade count: {trade_count}")
    return (total_main_pnl, total_hedge_pnl, trade_count)

def simulate_all_models(test_start_date='2022-01-01', test_end_date='2023-01-01'):
    df = pd.read_csv('output.csv')
    results = []
    for symbol in df['symbol'].unique():
        print(f"Simulating {symbol}...")
        (main, hedge, tradec) = simulate_model(symbol, test_start_date, test_end_date)
        results.append({'symbol': symbol,
                        'main pnl': main, 
                        'hedge pnl': hedge, 
                        'total pnl': main + hedge,
                        'trade count': tradec})
    pd.DataFrame(results).set_index('symbol').to_csv('simulation_results.csv')


In [230]:
#SP50 = get_SP(100)
#get_weekly_stock_bars(SP50['Symbol'], start=dt.datetime(2016, 1, 1))

In [231]:
#train_and_store_models()
#test_model('NVDA')
#simulate_model('LLY')
simulate_all_models()

Simulating AAPL...


C:\Users\maste\AppData\Local\Temp\ipykernel_18496\796046084.py:165: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[[-1.17887403e-01]
 [ 9.81252341e-01]
 [-1.25345282e+00]
 [-9.49709102e-01]
 [-1.66289932e-01]
 [ 3.99118050e-01]
 [ 7.58346314e-02]
 [ 1.45812753e+00]
 [-3.59709065e-01]
 [ 7.64978889e-01]
 [ 8.39801391e-01]
 [-4.97437314e-01]
 [ 1.22360032e-01]
 [-1.18506956e+00]
 [-3.35554811e+00]
 [-2.91165012e-01]
 [-8.12448635e-01]
 [ 1.18391845e+00]
 [ 1.22290847e+00]
 [-8.10292579e-01]
 [ 6.97034524e-02]
 [-1.12311129e+00]
 [-7.07614250e-01]
 [ 5.32882370e-01]
 [ 5.07316350e-02]
 [ 4.03911873e-01]
 [-1.89805543e-01]
 [ 1.28711239e+00]
 [ 7.96086851e-01]
 [ 5.00211302e-03]
 [ 1.20283788e-01]
 [-7.64148500e-01]
 [ 2.73503960e-02]
 [-1.31629122e+00]
 [ 2.70594911e+00]
 [-6.83627239e-01]
 [-8.44696727e-02]
 [ 6.75489968e-02]
 [ 6.51032422e-01]
 [-4.02914042e-01]
 [-8.28090165e-01]
 [-1.19097464e+00]
 [-2.72026062e-0

main pnl: -62.37218618659513
hedge pnl: 47.04730878406815
trade count: 2
Simulating ABBV...


C:\Users\maste\AppData\Local\Temp\ipykernel_18496\796046084.py:165: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[[-1.17887403e-01]
 [ 9.81252341e-01]
 [-1.25345282e+00]
 [-9.49709102e-01]
 [-1.66289932e-01]
 [ 3.99118050e-01]
 [ 7.58346314e-02]
 [ 1.45812753e+00]
 [-3.59709065e-01]
 [ 7.64978889e-01]
 [ 8.39801391e-01]
 [-4.97437314e-01]
 [ 1.22360032e-01]
 [-1.18506956e+00]
 [-3.35554811e+00]
 [-2.91165012e-01]
 [-8.12448635e-01]
 [ 1.18391845e+00]
 [ 1.22290847e+00]
 [-8.10292579e-01]
 [ 6.97034524e-02]
 [-1.12311129e+00]
 [-7.07614250e-01]
 [ 5.32882370e-01]
 [ 5.07316350e-02]
 [ 4.03911873e-01]
 [-1.89805543e-01]
 [ 1.28711239e+00]
 [ 7.96086851e-01]
 [ 5.00211302e-03]
 [ 1.20283788e-01]
 [-7.64148500e-01]
 [ 2.73503960e-02]
 [-1.31629122e+00]
 [ 2.70594911e+00]
 [-6.83627239e-01]
 [-8.44696727e-02]
 [ 6.75489968e-02]
 [ 6.51032422e-01]
 [-4.02914042e-01]
 [-8.28090165e-01]
 [-1.19097464e+00]
 [-2.72026062e-0

main pnl: -34.30801366023428
hedge pnl: 22.39215918526679
trade count: 3
Simulating ABT...


...

main pnl: -60.06364359586303
hedge pnl: -16.266482361248613
trade count: 1
Simulating WMT...


C:\Users\maste\AppData\Local\Temp\ipykernel_18496\796046084.py:165: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[[-1.17887403e-01]
 [ 9.81252341e-01]
 [-1.25345282e+00]
 [-9.49709102e-01]
 [-1.66289932e-01]
 [ 3.99118050e-01]
 [ 7.58346314e-02]
 [ 1.45812753e+00]
 [-3.59709065e-01]
 [ 7.64978889e-01]
 [ 8.39801391e-01]
 [-4.97437314e-01]
 [ 1.22360032e-01]
 [-1.18506956e+00]
 [-3.35554811e+00]
 [-2.91165012e-01]
 [-8.12448635e-01]
 [ 1.18391845e+00]
 [ 1.22290847e+00]
 [-8.10292579e-01]
 [ 6.97034524e-02]
 [-1.12311129e+00]
 [-7.07614250e-01]
 [ 5.32882370e-01]
 [ 5.07316350e-02]
 [ 4.03911873e-01]
 [-1.89805543e-01]
 [ 1.28711239e+00]
 [ 7.96086851e-01]
 [ 5.00211302e-03]
 [ 1.20283788e-01]
 [-7.64148500e-01]
 [ 2.73503960e-02]
 [-1.31629122e+00]
 [ 2.70594911e+00]
 [-6.83627239e-01]
 [-8.44696727e-02]
 [ 6.75489968e-02]
 [ 6.51032422e-01]
 [-4.02914042e-01]
 [-8.28090165e-01]
 [-1.19097464e+00]
 [-2.72026062e-0

main pnl: 12.610684857557118
hedge pnl: 25.099732098771426
trade count: 2
Simulating XOM...


C:\Users\maste\AppData\Local\Temp\ipykernel_18496\796046084.py:165: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[[-1.17887403e-01]
 [ 9.81252341e-01]
 [-1.25345282e+00]
 [-9.49709102e-01]
 [-1.66289932e-01]
 [ 3.99118050e-01]
 [ 7.58346314e-02]
 [ 1.45812753e+00]
 [-3.59709065e-01]
 [ 7.64978889e-01]
 [ 8.39801391e-01]
 [-4.97437314e-01]
 [ 1.22360032e-01]
 [-1.18506956e+00]
 [-3.35554811e+00]
 [-2.91165012e-01]
 [-8.12448635e-01]
 [ 1.18391845e+00]
 [ 1.22290847e+00]
 [-8.10292579e-01]
 [ 6.97034524e-02]
 [-1.12311129e+00]
 [-7.07614250e-01]
 [ 5.32882370e-01]
 [ 5.07316350e-02]
 [ 4.03911873e-01]
 [-1.89805543e-01]
 [ 1.28711239e+00]
 [ 7.96086851e-01]
 [ 5.00211302e-03]
 [ 1.20283788e-01]
 [-7.64148500e-01]
 [ 2.73503960e-02]
 [-1.31629122e+00]
 [ 2.70594911e+00]
 [-6.83627239e-01]
 [-8.44696727e-02]
 [ 6.75489968e-02]
 [ 6.51032422e-01]
 [-4.02914042e-01]
 [-8.28090165e-01]
 [-1.19097464e+00]
 [-2.72026062e-0

main pnl: 29.275487102701
hedge pnl: -13.640447402072326
trade count: 2


In [232]:
results = pd.read_csv('simulation_results.csv')
print(results['main pnl'].sum())
print(results['hedge pnl'].sum())

222.00489206130555
-506.0807836120941
